# Optimal Transport Algorithms Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from uot.analysis import get_agg_table, get_mean_comparison_table, get_std_comparison_table, display_mean_and_std, display_all_metrics

plt.style.use('ggplot')
sns.set_palette("colorblind")

In [4]:
results_df = pd.read_csv('result_2025-05-01_02-52-55.csv')

print(f"Shape of results: {results_df.shape}")
print(f"Algorithms: {results_df['name'].unique()}")
print(f"Datasets: {results_df['dataset'].unique()}")

results_df.head()

Shape of results: (5740, 34)
Algorithms: ['pot-lp' 'ott-jax-sinkhorn' 'jax-sinkhorn' 'optax-grad-ascent']
Datasets: ['32 1D gamma' '64 1D gamma' '256 1D gamma' '512 1D gamma' '1024 1D gamma'
 '2048 1D gamma' '32 1D gaussian' '64 1D gaussian' '256 1D gaussian'
 '512 1D gaussian' '1024 1D gaussian' '2048 1D gaussian' '32 1D beta'
 '64 1D beta' '256 1D beta' '512 1D beta' '1024 1D beta' '2048 1D beta'
 '32 1D beta_cauchy_gamma_gaussian' '64 1D beta_cauchy_gamma_gaussian'
 '128 1D beta_cauchy_gamma_gaussian' '256 1D beta_cauchy_gamma_gaussian'
 '512 1D beta_cauchy_gamma_gaussian' '1024 1D beta_cauchy_gamma_gaussian'
 '2048 1D beta_cauchy_gamma_gaussian' 'WhiteNoise 32x32'
 'CauchyDensity 32x32' 'GRFmoderate 32x32' 'GRFrough 32x32'
 'GRFsmooth 32x32' 'LogGRF 32x32' 'LogitGRF 32x32'
 'MicroscopyImages 32x32' 'Shapes 32x32' 'ClassicImages 32x32'
 '3D_Colored_Mesh_red_1024x1024pts' '3D_Colored_Mesh_red_2048x2048pts']


,Unnamed: 0,problem,source_measure_name,target_measure_name,source_gamma,target_gamma,dataset,time,cost_rerr,coupling_avg_err,...,source_color_name,source_num_points,target_mesh_name,target_color_channel,target_color_name,target_num_points,source_points,target_points,color_mode,name
0,3,Simple transport,32 1D gamma,32 1D gamma,"(np.float64(2.0), np.float64(6.0))","(np.float64(8.0), np.float64(2.72))",32 1D gamma,0.688336,1.557822e-16,1.516189e-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pot-lp
1,4,Simple transport,32 1D gamma,32 1D gamma,"(np.float64(2.0), np.float64(6.0))","(np.float64(2.0), np.float64(5.34))",32 1D gamma,0.574769,0.000000e+00,6.098637e-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pot-lp
2,5,Simple transport,32 1D gamma,32 1D gamma,"(np.float64(2.0), np.float64(6.0))","(np.float64(3.0), np.float64(1.41))",32 1D gamma,0.640088,2.884502e-16,3.684593e-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pot-lp
3,6,Simple transport,32 1D gamma,32 1D gamma,"(np.float64(2.0), np.float64(6.0))","(np.float64(5.0), np.float64(3.38))",32 1D gamma,0.579087,0.000000e+00,2.187992e-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pot-lp
4,7,Simple transport,32 1D gamma,32 1D gamma,"(np.float64(2.0), np.float64(6.0))","(np.float64(10.0), np.float64(4.03))",32 1D gamma,0.582283,0.000000e+00,1.365841e-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pot-lp


In [21]:
metrics_to_analyze = ['time', 'cost_rerr', 'coupling_avg_err']

for metric in metrics_to_analyze:
    print(f"\n## Analysis of {metric} ##")
    display_mean_and_std(results_df, metric)


## Analysis of time ##


,dataset,pot-lp,ott-jax-sinkhorn,jax-sinkhorn,optax-grad-ascent
0,32 1D gamma,0.594633,7.870374,5.226564,2.089505
1,64 1D gamma,0.790995,14.012934,10.026857,2.871272
2,256 1D gamma,6.355724,26.249865,14.483131,30.109811
3,512 1D gamma,22.816457,172.120468,82.834946,6.432182
4,1024 1D gamma,157.696580,957.398340,442.800824,21.845270
5,2048 1D gamma,534.489982,2441.057092,1029.648571,82.963372
6,32 1D gaussian,0.619189,20.695721,18.424764,3.535471
7,64 1D gaussian,0.895952,31.765082,24.441810,3.220959
8,256 1D gaussian,9.143174,76.938644,40.325279,23.011160
9,512 1D gaussian,51.896794,549.214355,246.651783,336.474754



## Analysis of cost_rerr ##


,dataset,pot-lp,ott-jax-sinkhorn,jax-sinkhorn,optax-grad-ascent
0,32 1D gamma,0.000000,0.808631,0.808130,0.940784
1,64 1D gamma,0.000000,0.349908,0.348742,0.997926
2,256 1D gamma,0.000000,2.551291,2.548138,0.998016
3,512 1D gamma,0.000000,1.672507,1.667075,1.000000
4,1024 1D gamma,0.000000,0.673603,0.667033,1.000000
5,2048 1D gamma,0.000000,22.585105,22.565076,1.000000
6,32 1D gaussian,0.000000,0.008820,0.008264,0.994718
7,64 1D gaussian,0.000000,0.012751,0.011964,0.998803
8,256 1D gaussian,0.000000,0.017849,0.015670,0.999338
9,512 1D gaussian,0.000000,0.037756,0.035161,0.999882



## Analysis of coupling_avg_err ##


,dataset,pot-lp,ott-jax-sinkhorn,jax-sinkhorn,optax-grad-ascent
0,32 1D gamma,0.000000,0.000336,0.000336,0.001096
1,64 1D gamma,0.000000,0.000216,0.000216,0.000264
2,256 1D gamma,0.000000,0.000024,0.000024,0.000016
3,512 1D gamma,0.000000,0.000007,0.000007,0.000004
4,1024 1D gamma,0.000000,0.000002,0.000002,0.000001
5,2048 1D gamma,0.000000,0.000000,0.000000,0.000000
6,32 1D gaussian,0.000000,0.000266,0.000266,0.001223
7,64 1D gaussian,0.000000,0.000191,0.000191,0.000269
8,256 1D gaussian,0.000000,0.000020,0.000020,0.000017
9,512 1D gaussian,0.000000,0.000007,0.000007,0.000004


In [5]:
one_dim_datasets = [ds for ds in results_df['dataset'].unique() if '1D' in ds]
two_dim_datasets = [ds for ds in results_df['dataset'].unique() if '2D' in ds or 'x32' in ds]
mesh_datasets = [ds for ds in results_df['dataset'].unique() if 'Mesh' in ds]

dataset_groups = {
    '1D Datasets': one_dim_datasets,
    '2D Datasets': two_dim_datasets,
    '3D Mesh Datasets': mesh_datasets
}

algorithms = results_df['name'].unique()

In [29]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

def plot_runtime_distributions(datasets, results_df, algorithms, save_path):
    os.makedirs(save_path, exist_ok=True)
    
    for dataset in datasets:
        dataset_data = results_df[results_df['dataset'] == dataset]
        
        fig, axes = plt.subplots(1, 2, figsize=(18, 5))

        ax1 = axes[0]
        ax1.set_yscale('log')
        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['time']
            if len(algo_data) > 0:
                sns.kdeplot(algo_data, ax=ax1, label=algo, fill=True, alpha=0.3)
        
        ax1.set_title(f'KDE of Runtime for {dataset}')
        ax1.set_xlabel('Runtime (seconds)')
        ax1.set_ylabel('Frequency')
        ax1.legend()

        ax2 = axes[1]
        plot_data = []
        labels = []

        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['time']
            if len(algo_data) > 0:
                plot_data.append(algo_data)
                labels.append(algo)
            else:
                print(f"Algorithm: {algo}, Dataset: {dataset}, Count: {len(algo_data)} - No data")

        ax2.boxplot(plot_data, tick_labels=labels, showmeans=True)
        ax2.set_title(f'Box Plot of Runtime for {dataset}')
        ax2.set_ylabel('Runtime (seconds)')
        ax2.set_yscale('log')
        plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

        plt.tight_layout()
        save_file = os.path.join(save_path, f"{dataset}_runtime_distributions.png")
        plt.savefig(save_file)
        plt.close(fig)


def plot_cost_err_distributions(datasets, results_df, algorithms, save_path):
    os.makedirs(save_path, exist_ok=True)

    for dataset in datasets:
        dataset_data = results_df[results_df['dataset'] == dataset]

        fig, axes = plt.subplots(1, 2, figsize=(18, 5))

        ax1 = axes[0]
        ax1.set_yscale('log')
        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['cost_rerr']
            if len(algo_data) > 1 and algo_data.nunique() > 1:
                sns.kdeplot(algo_data, ax=ax1, label=algo, fill=True, alpha=0.3)
            elif algo_data.nunique() == 1:
                print(f"Algorithm: {algo}, Dataset: {dataset}, Count: {len(algo_data)} - Constant value")
                const_val = algo_data.iloc[0]
                ax1.axvline(const_val, label=f"{algo} (constant)", linestyle='--')

        ax1.set_title(f'KDE of Cost Error for {dataset}')
        ax1.set_xlabel('Cost Error')
        ax1.set_ylabel('Frequency')
        ax1.legend()

        ax2 = axes[1]
        plot_data = []
        labels = []

        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['cost_rerr']
            if len(algo_data) > 0:
                plot_data.append(algo_data)
                labels.append(algo)

        ax2.boxplot(plot_data, tick_labels=labels, showmeans=True)
        ax2.set_title(f'Box Plot of Cost Error for {dataset}')
        ax2.set_ylabel('Cost Error')
        ax2.set_yscale('log')
        plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f"{dataset}_cost_err_distributions.png"))
        plt.close(fig)


def plot_coupling_err_distributions(datasets, results_df, algorithms, save_path):
    os.makedirs(save_path, exist_ok=True)

    for dataset in datasets:
        dataset_data = results_df[results_df['dataset'] == dataset]

        fig, axes = plt.subplots(1, 2, figsize=(18, 5))

        ax1 = axes[0]
        ax1.set_yscale('log')
        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['coupling_avg_err']
            if len(algo_data) > 0:
                sns.kdeplot(algo_data, ax=ax1, label=algo, fill=True, alpha=0.3)

        ax1.set_title(f'KDE of Coupling Avg. Error for {dataset}')
        ax1.set_xlabel('Coupling Avg. Error')
        ax1.set_ylabel('Frequency')
        ax1.legend()

        ax2 = axes[1]
        plot_data = []
        labels = []

        for algo in algorithms:
            algo_data = dataset_data[dataset_data['name'] == algo]['coupling_avg_err']
            if len(algo_data) > 0:
                plot_data.append(algo_data)
                labels.append(algo)

        ax2.boxplot(plot_data, tick_labels=labels, showmeans=True)
        ax2.set_title(f'Box Plot of Coupling Avg. Error for {dataset}')
        ax2.set_ylabel('Coupling Avg. Error')
        ax2.set_yscale('log')
        plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f"{dataset}_coupling_err_distributions.png"))
        plt.close(fig)




In [30]:
for group_name, datasets in dataset_groups.items():
    print(f"\n### {group_name} Runtime Analysis ###")
    plot_runtime_distributions(datasets, results_df, algorithms, save_path='runtime_distributions')
    print(f"\n### {group_name} Cost Error Analysis ###")
    plot_cost_err_distributions(datasets, results_df, algorithms, save_path='cost_err_distributions')
    print(f"\n### {group_name} Coupling Error Analysis ###")
    plot_coupling_err_distributions(datasets, results_df, algorithms, save_path='coupling_err_distributions')


### 1D Datasets Runtime Analysis ###

### 1D Datasets Cost Error Analysis ###

### 1D Datasets Cost Error Analysis ###
Algorithm: optax-grad-ascent, Dataset: 512 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 512 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 1024 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 1024 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 2048 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 2048 1D gamma, Count: 42 - Constant value
Algorithm: optax-grad-ascent, Dataset: 1024 1D beta_cauchy_gamma_gaussian, Count: 25 - Constant value
Algorithm: optax-grad-ascent, Dataset: 1024 1D beta_cauchy_gamma_gaussian, Count: 25 - Constant value
Algorithm: optax-grad-ascent, Dataset: 2048 1D beta_cauchy_gamma_gaussian, Count: 25 - Constant value
Algorithm: optax-grad-ascent, Dataset: 2048 1D beta_cauchy_gamma_gaussian, Count: 25 - Con